<p>
  <img src="https://upload.wikimedia.org/wikipedia/commons/f/fb/Escudo-UdeA.svg"
       alt="UdeA logo"
       height="190px"
       align="left"
       hspace="15px">
</p>

<h1 style="margin-top: 20px;"><b>5. Multimeter calibration</b></h1>

<hr>

<div align="right" style="font-size: 14px; line-height: 1.4;">
  <b>Estiven Castrillon Alzate</b><br>
  Institute of Physics<br>
  <i>Universidad de Antioquia</i><br>
  <span style="font-size: 12px;">Date: TO BE DEFINED</span>
</div>

<br><br>

# Table of Contents
1. [Introduction](#introduction)  
2. [Theory](#theory)  
1. [Analysis and code](#analysis-and-code)  
    - [1.1 Measurement error](#1.1.-measurement-error)
    - [1.2 Uncertainties analysis](#1.2.-uncertainties-analysis)
    - [1.3 Uncertainties report](#1.3.-uncertainties-report)
4. [Results](#results)  
5. [Conclusions](#conclusions)  
6. [References](#references)

In [1]:
import pandas as pd
import numpy as np

# data = pd.read_csv('~/datos_resistencia_ohms.csv', 
#                    delimiter=';', usecols=['resistencia_0 (ohm)', 'resistencia_47 (ohm)', 'resistencia_330 (ohm)'])
# data.to_json()

# Import of resistance calibration data
data_json = '{"resistencia_0 (ohm)":{"0":0.2,"1":0.3,"2":0.2,"3":0.2,"4":0.2},"resistencia_47 (ohm)":{"0":46.8,"1":46.7,"2":46.6,"3":46.6,"4":46.7},"resistencia_330 (ohm)":{"0":332.3,"1":332.1,"2":332.2,"3":331.8,"4":331.8}}'
data = pd.read_json(data_json)
data

/tmp/ipykernel_8948/1346333555.py:10: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_json(data_json)


,resistencia_0 (ohm),resistencia_47 (ohm),resistencia_330 (ohm)
0,0.2,46.8,332.3
1,0.3,46.7,332.1
2,0.2,46.6,332.2
3,0.2,46.6,331.8
4,0.2,46.7,331.8


Calculate the average value $R_x$ and the standard uncertainty $\alpha_x$.


In [2]:
means = round(data.mean(), 2)
standard_uncertainty = round(data.std() / np.sqrt(5), 2)

# Report of averages and standard uncertainties
print(f'Averages of the data: \n{means}\n')
print(f'Standard uncertainties of the data: \n{standard_uncertainty}')

Averages of the data: 
resistencia_0 (ohm)        0.22
resistencia_47 (ohm)      46.68
resistencia_330 (ohm)    332.04
dtype: float64

Standard uncertainties of the data: 
resistencia_0 (ohm)      0.02
resistencia_47 (ohm)     0.04
resistencia_330 (ohm)    0.10
dtype: float64


# 1. Analysis and code

## 1.1. Measurement error

Calculate the error in the measurement according to the following expression:

$$
E = (\bar{R_x} + \delta R_x) - (R_0 + \delta R_0) - (R_p + \delta R_p) \tag{1}
$$


In [3]:
means = round(data.mean(), 1)

# Definition of variables to apply equation (1)
multimeter_resolution = 0.1
resistor_resolution = 0.0002
zero_value = 0.3
resistor_tolerances = ['0%', '10%', '90%']
corrected_reference_values = [0, 47, 330]

errors = []

# Loop over all required values and apply equation (1)
for mean_value, corrected_value, tolerance in zip(means, corrected_reference_values, resistor_tolerances):
    error = (mean_value + multimeter_resolution) \
            - (zero_value + multimeter_resolution) \
            - (corrected_value + resistor_resolution)
    error = abs(round(error, 1))
    errors.append(error)
    
    # Report of mean value and error
    print(f'Measurement and error {tolerance}: {mean_value} ± {error}')

Measurement and error 0%: 0.2 ± 0.1
Measurement and error 10%: 46.7 ± 0.6
Measurement and error 90%: 332.0 ± 1.7


# 1.2. Uncertainties analysis

**Estimation of other uncertainties and of the combined uncertainty**

This section focuses on identifying additional sources of uncertainty present in the measurement process and combining them appropriately to obtain the total (combined) uncertainty associated with the measured quantity.

b) **Instrumental uncertainty of the standard** $u(\delta R_p)$: due to the tolerance given by the manufacturer, assuming a rectangular distribution.

$u(\delta R_p) = \frac{\delta R_p}{2\sqrt{3}}$

In [4]:
manufacturer_resistance_uncertainty = means * resistor_resolution
round(manufacturer_resistance_uncertainty / (2 * np.sqrt(3)), 5)

resistencia_0 (ohm)      0.00001
resistencia_47 (ohm)     0.00270
resistencia_330 (ohm)    0.01917
dtype: float64

**c) Multimeter resolution uncertainty $u(\delta R_x)$:** it is estimated assuming a rectangular distribution.

In [5]:
multimeter_resolution_uncertainty = multimeter_resolution / (2 * np.sqrt(3))
round(multimeter_resolution_uncertainty, 2)

np.float64(0.03)

**d) Combined uncertainty $u_c$:** it is obtained as the quadratic sum of all the uncertainties.

In [6]:
# Creation of an array with 3 values for the multimeter resolution
multimeter_resolution_uncert = np.arange(0, 3)
multimeter_resolution_uncert[:] = multimeter_resolution_uncertainty

compressed_errors = zip(resistor_tolerances, errors, multimeter_resolution_uncert, manufacturer_resistance_uncertainty)
combined_uncertainties = []

# Loop to compute the square root of the sum of squares of all uncertainties
for tolerance, error, uncert_multi, uncert_res in compressed_errors:
    combined_uncert = np.sqrt(error**2 + uncert_multi**2 + uncert_res**2)
    combined_uncert = round(combined_uncert, 2)
    combined_uncertainties.append(combined_uncert)
    print(f'For {tolerance}: {combined_uncert}')

For 0%: 0.1
For 10%: 0.6
For 90%: 1.7


**e) Expanded uncertainty $U$: assuming a normal distribution, report with a 95% confidence level the uncertainty obtained in item d.**}

In [7]:
# Calculation of expanded uncertainties using the updated variable names
expanded_uncertainty = combined_uncertainties
expanded_uncertainties = []
zip_expanded = zip(resistor_tolerances, combined_uncertainties)

# Compute expanded uncertainties
for tolerance, uncertainty in zip_expanded:
    expanded_uncertainties.append(uncertainty * 2)
    print(f'For {tolerance}: {uncertainty * 2}')


For 0%: 0.2
For 10%: 1.2
For 90%: 3.4


## 1.3. Uncertainties report
Report of the considered uncertainties:

In [8]:
# Report of the considered uncertainties
print("Instrumental uncertainty of the standard resistor (u(δRp)):")
for tolerance, uncertainty in zip(resistor_tolerances, manufacturer_resistance_uncertainty):
    print(f"For {tolerance}: {round(uncertainty, 5)} Ω")

print("\nInstrumental uncertainty of the multimeter (u(δRx)):")
print(round(multimeter_resolution_uncertainty, 2), "Ω")

print("\nCombined uncertainty (uc):")
for tolerance, uncertainty in zip(resistor_tolerances, combined_uncertainties):
    print(f"For {tolerance}: {uncertainty} Ω")

print("\nExpanded uncertainty (U) with 95% confidence:")
for tolerance, uncertainty in zip(resistor_tolerances, expanded_uncertainties):
    print(f"For {tolerance}: {uncertainty} Ω")


Instrumental uncertainty of the standard resistor (u(δRp)):
For 0%: 4e-05 Ω
For 10%: 0.00934 Ω
For 90%: 0.0664 Ω

Instrumental uncertainty of the multimeter (u(δRx)):
0.03 Ω

Combined uncertainty (uc):
For 0%: 0.1 Ω
For 10%: 0.6 Ω
For 90%: 1.7 Ω

Expanded uncertainty (U) with 95% confidence:
For 0%: 0.2 Ω
For 10%: 1.2 Ω
For 90%: 3.4 Ω


Expanded uncertainty for the table of literal 5

In [9]:
abs(abs(means - np.array(corrected_reference_values)) - np.array(expanded_uncertainties))

resistencia_0 (ohm)      0.0
resistencia_47 (ohm)     0.9
resistencia_330 (ohm)    1.4
dtype: float64